In [4]:
import pymongo
import os,sys
import dsa_mongo_common_functions as dsa
import cdsa_loader_helper_functions as cdsa_helpers

client = pymongo.MongoClient('localhost',27017)
db_ptr = client['TCGA_Pyramid_DB']
pyramid_root = '/PYRAMIDS/PYRAMIDS/CDSA/' 
reload(dsa)
reload(cdsa_helpers)

pyramid_db = client['CDSA_Pyramid_DB']


load_errors_db = client['CDSA_LoadErrors']
pyr_err_db = load_errors_db['Pyramid_Dups']

In [5]:
pyramid_collections = [x for x in os.listdir(pyramid_root) if os.path.isdir(os.path.join(pyramid_root,x)) ]
print len(pyramid_collections),'pyramid collection sets are available'
print pyramid_collections

65 pyramid collection sets are available
['DLBC_Frozen', 'OV_Frozen', 'ESCA_Frozen', 'TGCT_Frozen', 'CHOL_Diagnostic', 'LUSC_Frozen', 'HNSC_Diagnostic', 'PAAD_Diagnostic', 'UCEC_Diagnostic', 'BLCA_Diagnostic', 'BLCA_Frozen', 'UCS_Diagnostic', 'COAD_Frozen', 'KIRP_Diagnostic', 'DLBC_Diagnostic', 'LUAD_Diagnostic', 'LUAD_Frozen', 'UCS_Frozen', 'ESCA_Diagnostic', 'UVM_Frozen', 'THCA_Frozen', 'MESO_Diagnostic', 'CESC_Frozen', 'READ_Diagnostic', 'PRAD_Frozen', 'COAD_Diagnostic', 'THYM_Frozen', 'SKCM_Frozen', 'BRCA_Frozen', 'BRCA_Diagnostic', 'KIRC_Frozen', 'MESO_Frozen', 'PCPG_Diagnostic', 'READ_Frozen', 'THCA_Diagnostic', 'TGCT_Diagnostic', 'STAD_Diagnostic', 'THYM_Diagnostic', 'KIRP_Frozen', 'CHOL_Frozen', 'SKCM_Diagnostic', 'LGG_Diagnostic', 'SARC_Frozen', 'PRAD_Diagnostic', 'STAD_Frozen', 'GBM_Diagnostic', 'LGG_Frozen', 'LIHC_Frozen', 'GBM_DIAG_ANALYSIS_TIF', 'OV_Diagnostic', 'GBM_Frozen', 'UVM_Diagnostic', 'LIHC_Diagnostic', 'KICH_Frozen', 'UCEC_Frozen', 'ACC_Frozen', 'KICH_Diagnostic'

In [7]:
#collection_id='PRAD_Frozen'
#print pyramid_collections ## This lists all the top level folders
#pyr_err_db.drop()

In [8]:
for collection_id in pyramid_collections:
    curr_pyramid_slide_list = dsa.find_pyramid_images(collection_id,[pyramid_root])
    print "PROCESSING ",collection_id
    
    duplicated_pyramids = 0

    pyramids_processed = 0
    newly_processed = 0
    already_processed = 0
    #pyramid_db[collection_id].drop()

    for pyr in curr_pyramid_slide_list:
        pyramid_name = os.path.basename(pyr)
        qry = pyramid_db[collection_id].find_one( {'pyramid_name': pyramid_name})
        if not qry:
            fs = os.path.getsize(pyr)
            (openslide_could_open, width, height, filesize, orig_resolution, slide_name,md5, pyramid_properties) = cdsa_helpers.openslide_test_file_mongo( pyr, 'pyramid', client)

            pyramid_metadata = { 'pyramid_w_path': pyr, 'pyramid_name': pyramid_name, 'file_size':fs, 'width':width, 'height':height,
                                 'pyramid_properties': cdsa_helpers.clean_openslide_keys ( pyramid_properties), 'slide_md5': md5, 
                                 'conversion_info': 'TBD'
                                 }
                #print slide_metadata
            pyramid_db[collection_id].insert_one(pyramid_metadata)
            newly_processed +=1 
        else:
            ## Now I need to see if the file I am scanning is the same file already loaded, or if I inadvertently
            ## made multiple pyramids for the same svs file
            if qry['pyramid_w_path'] != pyr:
                img_size_in_db = os.path.getsize(pyr)
                new_img_size = os.path.getsize(qry['pyramid_w_path'])
                if img_size_in_db != new_img_size:
#                     print
#                     print qry['pyramid_w_path']
#                     print pyr
                    
                    pyr_err_db.insert_one( {'pyr_in_db': qry['pyramid_w_path'], 'pyr_thats_duped': pyr } )
                    duplicated_pyramids +=1
            already_processed +=1

        pyramids_processed +=1        
        output = "Total Processed: %d  Newly Processed: %d Dup Slides or Already Loaded: %d   Dup Pyramids %d" % (pyramids_processed, newly_processed, already_processed, duplicated_pyramids )
        dsa.LinePrinter(output)


Processed: 45 pyramidsPROCESSING  DLBC_Frozen
Processed: 2321 pyramidsPROCESSING  OV_Frozen
Processed: 248 pyramidsPROCESSING  ESCA_Frozen
Processed: 156 pyramidsPROCESSING  TGCT_Frozen
Processed: 36 pyramidsPROCESSING  CHOL_Diagnostic
Processed: 1395 pyramidsPROCESSING  LUSC_Frozen
Processed: 591 pyramidsPROCESSING  HNSC_Diagnostic
Processed: 58 pyramidsPROCESSING  PAAD_Diagnostic
Processed: 651 pyramidsPROCESSING  UCEC_Diagnostic
Processed: 545 pyramidsPROCESSING  BLCA_Diagnostic
Processed: 796 pyramidsPROCESSING  BLCA_Frozen
Processed: 95 pyramidsPROCESSING  UCS_Diagnostic
Processed: 1663 pyramidsPROCESSING  COAD_Frozen
Processed: 332 pyramidsPROCESSING  KIRP_Diagnostic
Processed: 30 pyramidsPROCESSING  DLBC_Diagnostic
Processed: 677 pyramidsPROCESSING  LUAD_Diagnostic
Processed: 1470 pyramidsPROCESSING  LUAD_Frozen
Processed: 63 pyramidsPROCESSING  UCS_Frozen
Processed: 163 pyramidsPROCESSING  ESCA_Diagnostic
Processed: 78 pyramidsPROCESSING  UVM_Frozen
Processed: 835 pyramidsPROCE

In [26]:
print sld_properties

{u'openslide.level[3].height': u'5723', u'openslide.level[8].width': u'249', u'openslide.level[4].width': u'3995', u'openslide.level[7].tile-width': u'256', u'openslide.level[4].height': u'2861', u'openslide.level[0].width': u'63922', u'openslide.level[7].tile-height': u'256', u'openslide.quickhash-1': u'2c8f9d303053733a664830e0fbf4ecd09d11eabdef2d7e17b10704b21b5323e1', u'openslide.level[6].tile-height': u'256', u'openslide.level[2].height': u'11447', u'openslide.level[1].height': u'22894', u'openslide.level[4].tile-height': u'256', u'openslide.level[3].width': u'7990', u'openslide.level[1].width': u'31961', u'openslide.level[5].height': u'1430', u'openslide.level[3].tile-width': u'256', u'openslide.level[2].width': u'15980', u'openslide.level[7].height': u'357', u'openslide.level[0].tile-height': u'256', u'openslide.level[5].tile-height': u'256', u'openslide.level-count': u'9', u'tiff.ResolutionUnit': u'centimeter', u'openslide.level[0].height': u'45788', u'tiff.YResolution': u'10', u